In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the dataset
df = pd.read_csv('Salary Data.csv')

In [3]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [4]:
df.tail()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
370,35.0,Female,Bachelor's,Senior Marketing Analyst,8.0,85000.0
371,43.0,Male,Master's,Director of Operations,19.0,170000.0
372,29.0,Female,Bachelor's,Junior Project Manager,2.0,40000.0
373,34.0,Male,Bachelor's,Senior Operations Coordinator,7.0,90000.0
374,44.0,Female,PhD,Senior Business Analyst,15.0,150000.0


In [9]:
# Handle missing values by dropping rows with missing data
df_clean = df.dropna()

In [10]:
# Binning salaries into categories (e.g., Low, Medium, High)
salary_bins = [0, 50000, 100000, float('inf')]
salary_labels = ['Low', 'Medium', 'High']
df_clean['Salary Category'] = pd.cut(df_clean['Salary'], bins=salary_bins, labels=salary_labels)

<ipython-input-10-a596fb731b4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Salary Category'] = pd.cut(df_clean['Salary'], bins=salary_bins, labels=salary_labels)


In [11]:
# Define features and target
X = df_clean.drop(columns=['Salary', 'Salary Category'])
y = df_clean['Salary Category']

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
from sklearn.preprocessing import StandardScaler
# Define features
numerical_features = ['Age', 'Years of Experience']
categorical_features = ['Gender', 'Education Level', 'Job Title']

# Define the numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define the categorical transformer pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the transformers into a preprocessor object
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [36]:
# Define the model
clf = RandomForestClassifier(random_state=0)

In [37]:
from sklearn.pipeline import Pipeline
# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', clf)])

In [38]:
# Fit the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age',
                                                   'Years of Experience']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Education Level',
                                                   'Job Title'])])),
                ('model', RandomForestClassifier(random_state=0))])

In [39]:
# Make predictions
predictions = pipeline.predict(X_test)
print(predictions)

['High' 'Medium' 'High' 'Medium' 'High' 'High' 'High' 'High' 'Low'
 'Medium' 'High' 'High' 'Low' 'Medium' 'High' 'Low' 'Medium' 'Low'
 'Medium' 'High' 'High' 'High' 'High' 'Low' 'Medium' 'Medium' 'High' 'Low'
 'High' 'High' 'Low' 'High' 'Low' 'High' 'High' 'Low' 'High' 'Low' 'High'
 'Medium' 'Low' 'Medium' 'High' 'High' 'Low' 'High' 'High' 'Low' 'Low'
 'Low' 'High' 'Medium' 'High' 'Medium' 'High' 'High' 'High' 'Medium'
 'High' 'Medium' 'High' 'High' 'High' 'Medium' 'Low' 'Low' 'Low' 'High'
 'Low' 'Low' 'Medium' 'High' 'Medium' 'High' 'Medium']


In [40]:
pipeline.score(X_test, y_test)

0.8533333333333334